<a href="https://colab.research.google.com/github/VishnuKunchur/cosmic-aincounters/blob/main/Cosmic_AIncounters_Multi_Agentic_Simulations_of_a_Competitive_Strategy_Game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install -q langchain langchain-google-genai pydantic

###🪐 Cosmic AIncounters — a wild and chaotic game of alliances, negotiations, and warfare between LLM Agents!
👾 Each player is an alien in a troubled galaxy


Our game has three competitors: ALPHA 🔴𝛼, BETA 🔵𝛽, and GAMMA🟢𝛾 - each one a Gemini-based agent. Each of these agents will compete with the others in the game.


🌌<u>Goal</u>: establish 5 colonies on other players’ planets (foreign colonies)

🚀On each turn, one Offender picks a target and launches ships to invade the Defender's planet - unless they are able to negotiate. The other agent is a Bystander.

⚡Both Offender and Defender play a Cosmic card 🌀(to attack or to negotiate).

⚔️The Offender commits a certain number of ships from their established colonies to the face-off.

🛡️The Defender defends with the ships they have on the planet.

💥Upon winning, attackers gain foreign colonies, defenders hold strong.

🤝The Offender and Defender may also just choose to negotiate!

➕Any Bystander in the standoff may be invited to the party! Allies can choose to join either Offender or Defender (given an invite) for rewards.

🎯First player to reach 5 foreign colonies wins control of the galaxy!


<b>ENCOUNTER RESULT</b>:

⚔️<b>OFFENSE</b>: The number of ships committed to the attack by the Offender is added to the value of their Attack Card. If the Ally joins, their count of ships is added to the total.

🛡️<b>DEFENSE</b>: The number of ships remaining on the Defender's home planet colony is added to the value of their Attack Card. If the Ally joins, their count of ships is added to the total.

🤝Negotiate cards have 0 Attack value, but sparring Agents may choose to Negotiate a truce before the Attack is opened up to the galaxy and prevent any damage with mutual agreement.

In [15]:
import os
import sys
import time
from collections import defaultdict
import random
from typing import List, Dict, Optional
from enum import Enum
import json
import numpy as np
import pprint
# Third-party imports
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, field_validator, model_validator
from uuid import uuid4

## <b>ENTER YOUR GOOGLE API KEY HERE:</b>

In [16]:
# --- CONFIGURATION ---
# You will need a Google API key to use the Gemini model. Create one for free here: https://aistudio.google.com/app/api-keys
os.environ["GOOGLE_API_KEY"] = "" # always keep secret and remove before code-commits for security

UNIVERSE SETUP:

👾: Each player 🔴𝛼, 🔵𝛽, and 🟢𝛾 is configured with a home-planet system consisting of five (5) planets 🪐, each planet hosting a home colony of four (4) ships 🚀. Thus, each player has 20 ships. The Victory Point counter is set to 0 for each player.

--------------------Ō--------Ō------- Ō--------------------

----- 🎖️-🎖️-🎖️ VICTORY POINT COUNTER 🎖️-🎖️-🎖️ -----

🔴𝛼 : 0

🔵𝛽 : 0

🟢𝛾 : 0

--------------------Ō--------Ō------- Ō--------------------

WARP: The WARP, an interstellar junkyard that stores all ships destroyed in cosmic encounters, is also initialized.

In [17]:
def init_universe():
  # 3 player agents will compete
  players = ['ALPHA', 'BETA', 'GAMMA']
  planets = ['Planet-1', 'Planet-2', 'Planet-3', 'Planet-4', 'Planet-5']
  # """player -> planet -> occupier -> ship count (int)"""
  player_planet_occupancy = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

  # initialize universe
  for player in players:
    for planet in planets:
      for occupier in players:
        if player == occupier:
          # add four ships to each player's home planet
          player_planet_occupancy[player][planet][occupier] = 4
        else:
          player_planet_occupancy[player][planet][occupier] = 0

  # number of ships in the warp and victory point counter initialization
  warp_ship_count = dict([(player, 0) for player in players])
  player_vp_count = dict([(player, 0) for player in players])
  return players, planets, player_planet_occupancy, warp_ship_count, player_vp_count

In [18]:
# the player_planet_occupancy is flattened to minimize hallucination when this data is sent to the LLM
# reduce sparsity, only communicate occupied colonies
def get_occupancy_status(player_planet_occupancy) -> str:
  """
  generate a human and LLM-readable planetary occupancy status from session objects
  """
  occupancy_status = []
  for player, planet_occupancy in player_planet_occupancy.items():
    for planet, occupiers in planet_occupancy.items():
      for occupier, ships in occupiers.items():
        if ships != 0:
          occupancy_status.append({'Player': player, 'Planet': planet, 'Occupier': occupier, 'Ships': ships})
  occupancy_status = '\n'.join([str(o) for o in occupancy_status])
  return occupancy_status

COSMIC CARDS

In [19]:
class CosmicCardType(Enum):
  ATTACK = "Attack"
  NEGOTIATE = "Negotiate"

In [20]:
class CosmicCard(BaseModel):
    """
    represents a Cosmic card after pydantic validation
    """
    id: str
    type: CosmicCardType
    value: int

    @model_validator(mode="after")
    def post_coercion_checks(self):
        self._max_value = 61
        if self.type not in list(CosmicCardType):
          raise ValueError(f"Invalid card type {self.type}")
        if (self.type == CosmicCardType.NEGOTIATE) & (self.value != 0):
          raise ValueError("Negotiate cards have a value of 0")
        if (self.type == CosmicCardType.ATTACK) & ((self.value < 0) | (self.value > self._max_value)):
          raise ValueError(f"Attack cards must have an integer value between 0 and {self._max_value}")
        return self

    def __str__(self):
      """readable representation of the CosmicCard"""
      return f"{self.id}: {self.type.value} ({self.value})"

In [21]:
class CosmicCardDeck():
  """
  create, prepare, and maintain a deck of Cosmic cards for use in the game.
  """

  def __init__(self):
    """
    builds and shuffles the Cosmic card deck.
    """
    self.deck = []
    self._build()
    self.shuffle()
    self.discard_pile = []

  def _build(self):
    """
    builds the Cosmic card deck.
    """
    if len(self.deck) == 0:
      # simulating a simple card pack with skewed attack values:
      attack_card_values = [0]*8 + list(range(1, 4))*4 + list(range(6, 10))*3 + list(range(15, 21))*2 + list(range(35, 41))*1 + [60]
      for value in attack_card_values:
        # add Attack cards to the deck:
        self.deck.append(CosmicCard(id=str(uuid4()).split('-')[0], type=CosmicCardType.ATTACK, value=value))
      # add 10 Negotiate cards to the deck:
      for _ in range(10):
        self.deck.append(CosmicCard(id=str(uuid4()).split('-')[0], type=CosmicCardType.NEGOTIATE, value=0))
    else:
      print("cannot rebuild a pre-existing deck! the present deck must be exhausted first.")
      pass

  def shuffle(self):
    """
    shuffles the Cosmic card deck.
    """
    # shuffle the built deck
    random.shuffle(self.deck)

  def deal(self, num_cards=1) -> List[CosmicCard]:
    """
    deals a specified number of cards from the deck (default = 1).
    """
    dealt_cards = []
    if len(self.deck) < num_cards:
      raise ValueError("insufficent cards in the deck.")
    for _ in range(num_cards):
      dealt_cards.append(self.deck.pop())
    return dealt_cards

  def add_to_discard_pile(self, cards: List[CosmicCard]):
    """
    adds a list of Cosmic cards to the discard_pile
    """
    self.discard_pile += cards

  def __str__(self):
    """
    returns a string representation of the Cosmic card deck.
    """
    return ', '.join([str(card) for card in self.deck])

  def __len__(self):
    """
    returns the length of the Cosmic card deck.
    """
    return len(self.deck)

AGENTIC AI - Firstly, what makes the present framework "Agentic"?  


Perception: The LLM Agent is able to obtain the state of the universe from a fully deterministic Game Engine, or main game loop that tracks and communicates the events in the progression of the game. The Agents receive instructions from and communicate with the Game Engine to learn about the state of the universe.

Autonomy, Agency, and Reasoning: We aim to design agents that are fully compliant with an existing, deterministic playbook and still have the "agency" to make ally, negotiate, or attack decisions autonomously based on their current perception of the state of the universe.

Memory: Ideally an Agent must have a comprehensive memory of all prior encounters.

A `PlayerAgent` can:
1) Select the best Cosmic card from their hand
2) Choose to attack, negotiate, propose or accept an alliance to join the encounter
3) Select the planet of the defender to attack, and the specific ships that are engaged for the encounter

The `PlayerAgent` must thus be able to:
1) Perceive the state of the universe: communicate with the Game Engine to send statements out to opponents and allies
2) Refer to the rulebook if necessary: Since the entire "rulebook" of the simplified game version is terse enough, I just stick it in the system prompt, so the entire information is available to the LLM for each `.decide()` call.
3) Plan and implement legal action(s): some validation implemented within the `Pydantic` models
4) Retain memory of past encounters: To keep the input token costs lowAgentic memory is currently disabled. It would be interesting to see how the Agents would respond if they were equipped with memory - would they tend to hallucinate more or less, etc.

Without providing any explicit strategy on how to win: how will the Agents respond and interact with each other?

Parsing LLM output with schema enforcement enables improved game and logical consistency with the deterministic Game Engine. This is achieved by implementing various game components as `Pydantic` models.

In [22]:
# Pydantic models:
class CommitmentEntry(BaseModel):
  Player: str = Field(description="The player whose home planetary system this planet is in")
  Planet: str = Field(description="The planet from which ships in your colony are being committed")
  Ships: int = Field(description="The number of ships committed from this colony")

  @field_validator('Ships')
  @classmethod
  def check_ships_range(cls, v):
      if not (1 <= v <= 4):
          raise ValueError('Number of ships committed from a single planet must be between 1 and 4.')
      return v

# invited allies - v0: just one
class DesiredAllyEntry(BaseModel):
  #since this is a 3-player game, only one entry is permitted.
  Player: str = Field(description="The player you want to ally with")

# Main-Player Agent Move Pydantic Model (Offender or Defender)
class MainAgentMove(BaseModel):
  """
  pydantic model for the Main Agent's move: Offender or Defender response
  """
  internal_thought: str = Field(description="Your strategic reasoning for this move. Keep it secret.")
  public_statement: str = Field(description="What you say to the other players (bluffing allowed).")
  selected_card_id: str = Field(description="The ID of the card you want to play. This must be a card in your current hand. Return only the card_id (8 characters)")
  selected_planet_id: Optional[str] = Field(description="The ID of the planet you want to invade as the Offender (if applicable). If you are the Defender, this should not be provided. Possible values are Planet-1, Planet-2, Planet-3, Planet-4, Planet-5")
  commitments: Optional[List[CommitmentEntry]] = Field(description="Breakdown of total ships you wish to commit in the JSON format. If you are the Offender this is a mandatory response. If you are the Defender, this should not be provided.")
  action_choice: str = Field(description="The specific action chosen from the available choices. (ATTACK / NEGOTIATE)")
  desired_ally: Optional[DesiredAllyEntry] = Field(description="""The player you want to ally with in JSON format. As the Offender/Defender; respond with the name of the Bystander you may wish to invite as an Ally (if applicable).
                                                                  If you do not want to call an Ally, this should not be provided.
                                                                  No self-invites or cross-invites between Offender/Defender.""")

  @model_validator(mode="after")
  def check_commitments_total(self):
    if self.commitments:
        total_committed_ships = sum(entry.Ships for entry in self.commitments)
        if not (1 <= total_committed_ships <= 4):
            raise ValueError(f"Total committed ships must be between 1 and 4, but got {total_committed_ships}.")
    return self


# Bystander Agent Move Pydantic Model
class BystanderAgentMove(BaseModel):
  """
  pydantic model for the Bystander Agent's move
  """
  internal_thought: str = Field(description="Your strategic reasoning for this move. Keep it secret.")
  public_statement: str = Field(description="What you say to the other players.")
  commitments: Optional[List[CommitmentEntry]] = Field(description="Breakdown of total ships you wish to commit in the JSON format. If you are a Bystander who has accepted an alliance, this is a mandatory response and you must commit atleast one ship.")
  action_choice: str = Field(description="The specific action chosen from the available choices. (ALLY / ABSTAIN)")
  desired_ally: Optional[DesiredAllyEntry] = Field(description="""The player you want to ally with. As the Bystander; provide the name of the Offender/Defender you accept as an Ally, given that they invited you to Ally (if applicable).
                                                                  If you do not want to Ally, this should not be provided.""")

  @model_validator(mode="after")
  def check_commitments_total(self):
    if self.commitments:
        total_committed_ships = sum(entry.Ships for entry in self.commitments)
        if not (1 <= total_committed_ships <= 4):
            raise ValueError(f"Total committed ships must be between 1 and 4, but got {total_committed_ships}.")
    return self

`PlayerAgent`

In [23]:
class PlayerAgent:
  """
  an Agentic AI Player in the game
  """
  def __init__(self, player, occupancy_status, warp_ship_count, player_vp_count, hand=[], model_name="gemini-2.5-flash"):
    # one of ALPHA, BETA, or GAMMA
    self.player = player
    # whether the player is either an Offender or a Defender. defaults to False (Bystander).
    self._main_player = False
    self.hand = hand
    self.occupancy_status = occupancy_status
    self.warp_ship_count = warp_ship_count
    self.player_vp_count = player_vp_count
    # initialize the LLM
    self.llm = ChatGoogleGenerativeAI(model=model_name, temperature=0.25)
    # configure Pydantic parser based on whether its a main player or a bystander
    self.update_parser()

  @property
  def main_player(self):
    return self._main_player

  @main_player.setter
  def main_player(self, value):
    if self._main_player != value:
        self._main_player = value
        self.update_parser()

  def update_parser(self):
    """
    depending on whether the Player is an Offender/Defender or Bystander, the Pydantic model in the expected output is modified
    """
    if self._main_player:
      self.parser = PydanticOutputParser(pydantic_object=MainAgentMove)
    else:
      self.parser = PydanticOutputParser(pydantic_object=BystanderAgentMove)

  def receive_cards(self, cards: List[CosmicCard]):
    self.hand += cards

  def get_hand(self) -> str:
    """
    human and LLM-readable hand representation
    """
    return ', '.join([str(card) for card in self.hand])

  def shuffle_hand(self) -> str:
    random.shuffle(self.hand)

  def deal(self, num_cards=1):
    """
    deals a specified number of cards from the deck (default = 1).
    if num_cards exceeds the number of cards in the deck, all cards in the deck are dealt.
    """
    dealt_cards = []
    if len(self.hand) == 0:
      raise ValueError("no cards left in the hand!")
    for _ in range(min(num_cards, len(self.hand))):
      dealt_cards.append(self.hand.pop())
    return dealt_cards

  def decide(self, game_context: str, valid_options: List[str]) -> MainAgentMove | BystanderAgentMove:
    """
    decision-making by the Agent based on the current game state

    @returns:
    pydantic model object with specified schema (MainAgentMove or BystanderAgentMove)
    """
    system_prompt = """
    You are a player in a game that is a simplified version of the popular Cosmic Encounter board game. Only use the information provided to make decisions.
    You belong to the legendary {player} race, one of three ancient galactic races (ALPHA, BETA, and GAMMA), each controlling a planetary system in the Ursa Macatrea galaxy.

    The players are struggling for cosmic supremacy and are attempting to colonize planets in foreign systems (planets belonging to other players).
    The winner of the game is the first player to establish five colonies on foreign planets.
    Each foreign colony gains the player a Victory Point. The loss of a colony on a planet reduces the player's Victory Point count by one.
    The player(s) who first get(s) to five (5) Victory Points (establishing 5 foreign colonies) win(s) the game.

    Throughout the game, you will be communicating with the Game Engine and with other players directly.

    HOW TO WIN: Establish five (5) colonies on foreign planets. You may establish only one colony on a planet. A colony is established by landing atleast one of your ships
    on the other player's planet, i.e., any planet outside of your home planetary system. The colony may be established via negotiation, diplomacy, or wars (encounters)
    with the other players. A player does not need to have colonies in all of the systems, just colonies on five planets outside their home system. These colonies
    may all be in one system or scattered over multiple systems. If two or more players establish their fifth foreign colony at the same time, they win together.

    PLAYERS: The game proceeds turn-by-turn in "encounters", with one player selected as the Offender, another player as the Defender, and all other players being Bystanders. If a Bystander mutually agrees to ally with either the Offender or Defender, they become an Ally for that encounter. Alliances are only agreed upon for the current encounter.

    Offender: The player whose turn it currently is
    Defender: The player who was destined to have a cosmic encounter by the Offender
    Bystander: Any player who is neither the Offender nor the Defender
    Ally: Any player who was invited to and accepted the invitation to the encounter with either the Offender or Defender. An Ally may only team up with one player. You cannot ally simultaneously with both the Offender and the Defender in the current encounter!

    OCCUPANCY: The current universe state (players, planets, occupiers, ships) is provided to you as a list of dictionaries:
    Example planet occupancy entry - BETA has a foreign colony on ALPHA's Planet-2 with 3 ships:

    "Player": "ALPHA", "Planet": "Planet-2", "Occupier": "BETA", "Ships": 3

    - Player: the player whose home planetary system this planet is in (ALPHA)
    - Planet: the planet_id (Planet-2)
    - Occupier: the player who owns a colony on this planet (BETA)
    - Ships: the number of ships in the colony the Occupier has on this planet (3)

    ITEMS: The following is a list of items that are used in the game -
    1) Planets: Each player has 5 planets in their home system. Players attempt to settle colonies (1 or more ships) on other players' planets (foreign planets or foreign systems)
    to win. Each player's planets are labeled as Planet-1, Planet-2, Planet-3, Planet-4, Planet-5. As the offender, you must specify the planet of the player you are having an encounter with (ex: Planet-4).
    2) Ships: The smallest unit of a colony, the presence of even one ship on a planet represents a colony. Each player starts with 20 ships, 4 on each planet.
    3) Colony: A colony is a fleet of one or more ships that are on a planet. A colony established on another player's planetary system is called a foreign colony. All ships on a planet that belong to a player count as one colony for that player.
    4) Warp: A central interstellar junkyard containing ships that were wrecked in cosmic encounters. All your ships that were lost in encounters end up here.
    5) Hyperspace Gate (Gate): A galactic wormhole that points to the planet (in the defender's system) under attack by the offending player. If you are the offender, the Gate sends your ships
    to a specific planet in the planetary system of the player you were destined to have an encounter with.
    6) Cosmic Cards: Cosmic cards may either be 'Attack' cards or 'Negotiate' cards. At the start of the game, each player starts with a hand of 8 randomly dealt Cosmic cards.
    These cards are played during encounters. Your Cosmic card deck is visible only to you and the Game Engine, unless stated otherwise. Each Cosmic card is represented as a string that follows the format:
    'card_id: card_type (attack_value)'. Cosmic cards in the hand are separated by a ','.

    The following is an example of a player's hand of Cosmic cards, along with an explanation:
    '9075822a: Attack (2), 23rg35f3: Attack (2), 438rhbfj: Attack(8), 09yb5f4h: Attack(8), 24v24v42: Attack(0), 32fq3fva: Negotiate (0)'

    For the first card in the hand, i.e, '9075822a: Attack (2)'
    card_id = 9075822a
    card_type = Attack
    attack_value = 2

    For the last card in the hand, i.e., '32fq3fva: Negotiate (0)'
    card_id = 32fq3fva
    card_type = Negotiate
    attack_value = 0

    To play a card during an encounter, you must specify the correct card_id (if applicable) in your response. You must select and respond with one card_id (8 character long string, ex: 9075822a) that is part of your current hand. Make sure to return only the card_id (ex: 9075822a) and not the entire card.

    RULES: The game proceeds in three phases: Setup, Destiny, and Encounter. The Destiny and Encounter phases occur turn by turn for each player, till one player has established five foreign colonies.

    1) Setup: Each player gets 5 planets and 20 ships. 4 ships are placed on each planet. The Cosmic card deck is shuffled and each player gets a hand of 8 cards.
     The remainder of the deck is depleted sequentially over turns or encounters. One player is selected to play first at random. This is the offensive player or Offender.
     Then,
    2) Destiny: One of the other two players is chosen to have a cosmic encounter with the offensive player at random. The randomly chosen player is the defensive player
or Defender. The encounter will take place in the Defender's home system.

    The Offender points the Hyperspace Gate to one planet in the Defender's system. When you are the Offender, you will be asked to specify the planet (Planet-1, Planet-2,
     .., Planet-5) in the Defender's system. When you are the Defender, one of your planets may be under attack (determined by the offensive player).
    If you are already an Occupier on a planet, you may select that planet to attack again but this does not give you an extra foreign colony or Victory Point. It only increases the count of ships in your existing colony on that planet, if you win the encounter.

    3) Encounter: The Offender commits one to four ships from any of their colonies. The offense may take ships from their home planet colonies or foreign colonies. Ships may
    all be taken from the same colony or from different colonies. A player should be careful not to remove all of the ships from a colony, however, as they will lose the colony by doing so.
    The offender may not choose more than four (4) ships from their colonies to attack. The defender may not add or subtract ships from the targeted planet.
    Bystander ships on the planet (ships in colonies owned by other players) do not add to the defense. Note that, in a home system, the defense may not have any ships on the targeted planet, in which case they defend the planet with zero ships.

    Cosmic card Selection: The Offender and the Defender now each select the best and most appropriate Cosmic card from their hand (an Attack or Negotiate) - but keep it secret and an internal thought
    - it is not revealed to other players immediately. The selected card (card_id) is only communicated to the Game Engine, unless specified otherwise.

    Calling Allies: Both the Offender and the Defender may choose to invite a Bystander in the encounter to become their Ally. The Offender and Defender must not invite each others as an Ally. Only a Bystander in the current encounter may be invited as an Ally.
    The alliances are only valid for the current encounter. Any invited Bystanders may choose to ally with or abstain from forming alliances with the Offender or Defender. If the Bystander accepts an invite, they form an alliance strictly with the Offender or strictly with the Defender for the current encounter.

    Alliances: The Ally then proceeds to commit one to four ships from any of their colonies. The ship selection to form and respond with "commitments" by the Ally follow the same rules as the Offender's ship selection and commitments described previously.
    The Ally helps the Offender or Defender only by contributing ships to the encounter and boosting the attack value. Allies do not play any cards during the encounter.

    Negotiation: Both Offender and Defender may attempt to negotiate by playing a Negotiate card for a peaceful resolution. A negotiation is successful only when both the Offender and Defender play a Negotiate card
    or an Attack (0) card. Attack (0) cards serve as Negotiate cards. Note that it may be possible for a player to claim that they are playing a Negotiate card but end up playing an Attack card instead.

    Negotiation Compensation: If one player plays Negotiate while the other plays an Attack card, the player of the Attack card automatically wins the encounter (unless they played an Attack (0) card, which is treated like a Negotiate card). The player who played Negotiate receives compensation in this case. For each ship lost to the warp, one Cosmic card is taken from the attacking player's hand.

    Offender and Ally Commitments: As the offender, if you attack, or as the ally, if you wish to side with an offender or defender, you must specify the number of ships you want to commit to the encounter by providing a "commitments" output in your response. First,
    decide the total number of ships you want to commit. Then identify the Player, Planet, and Ships from each entry in the planetary occupancy status where you are an Occupier from which you will send ships for the encounter.
    Using the provided planetary occupancy status:

    - Identify the planets where you are an Occupier (i.e. have a colony, either home colony or foreign colony)
    - Choose the number of ships from the colony to commit to the encounter (if desired).
    - You may only commit ships from a planet where you are an Occupier.
    - You must specify from which planet(s) and which player(s) you are taking ships.
    - You cannot commit more ships from a planet than are present in your colony there.
    - You may not commit more than 4 ships in total to the encounter across all planets and colonies.
    - When committing ships from a colony, make sure you leave atleast one ship behind to prevent loss of a Victory Point.

    Respond ONLY with valid JSON, in the following example format:

    "commitments": [

        "Player": "ALPHA",
        "Planet": "Planet-2",
        "Ships": 1
      ,

        "Player": "BETA",
        "Planet": "Planet-1",
        "Ships": 2

    ]

    WINNING THE ENCOUNTER: First, the total number of attack points for both the offense and defense is calculated.

    Assuming no Allies:
    Offender's Attack Points: The number of committed ships (1 to 4) plus the value of the Attack card played by the Offender.
    Defender's Attack Points: The number of ships on the planet under attack plus the value of the Attack card played by the Defender.

    Assuming Allies sided with the Offender:
    Offender's Attack Points: The number of committed ships (1 to 4) plus the value of the Attack card played by the Offender. The number of ships committed (1 to 4) by any offensive Allies is added to make the offense's total.
    Defender's Attack Points: The number of ships on the planet under attack plus the value of the Attack card played by the Defender.

    Assuming Allies sided with the Defender:
    Offender's Attack Points: The number of committed ships (1 to 4) plus the value of the Attack card played by the offender.
    Defender's Attack Points: The number of ships on the planet under attack plus the value of the Attack card played by the defender. The number of ships committed (1 to 4) by any defensive Allies is added to make the defence's total.

    Example 1:
    Offender (ALPHA): Committed 3 ships and played an Attack card with value 4. Total attack points = 3 + 4 = 7.
    Defender (BETA): Had 1 ship on the planet under attack and played an Attack card with value 9. Total attack points = 1 + 9 = 10.

    Defender BETA has more attack points than the offender ALPHA, so BETA wins this encounter and gets to keep all their ships on the planet under attack. All 3 of ALPHA's committed ships are sent to the warp. There were no Allies in this encounter.

    Example 2:
    Offender (GAMMA): Committed 1 ship and played an Attack card with value 10. Total attack points = 1 + 10 = 11.
    Defender (ALPHA): Had 4 ships on the planet under attack and played a Negotiate card that has value 0. Total attack points = 4 + 0 = 4.

    GAMMA wins this encounter, foiling ALPHA's negotiation bid, and keeps the 1 committed ship on the selected planet in ALPHA's system. All 4 of ALPHA's ships on the planet are sent to the warp. GAMMA gets a foreign colony. They advance by one Victory Point. Since ALPHA played a Negotiate card while GAMMA chose to attack, ALPHA receives compensation from GAMMA's deck. There were no Allies in this encounter.

    Example 3:
    Offender (BETA): Committed 4 ships and played an Attack card with value 20. Total attack points = 4 + 20 = 24.
    Defender (GAMMA): Had 2 ships on the planet under attack and played an Attack card with value 39. ALPHA joined as a defensive Ally to GAMMA's encounter with 2 ships. Total attack points = 2 + 39 + 2 = 43.

    GAMMA wins this encounter overwhelmingly with ALPHA's defensive support, resisting BETA's offense, and gets to keep all their ships on the planet under attack. All 4 of BETA's ships are sent to the warp. ALPHA gets 2 Cosmic cards from the deck as reward for co-winning the encounter as they committed 2 ships. ALPHA's ships return to their colonies.

    Example 4:
    Offender (ALPHA): Committed 3 ships and played an Attack (4) card. BETA joined as an offensive Ally to bolster ALPHA's attack with 1 ship, hoping to gain a Victory Point. Total attack points = 3 + 4 + 1 = 8.
    Defender (GAMMA): Had 0 ships on the planet under attack and played an Attack (7) card. Total attack points = 0 + 7 = 7.

    ALPHA and BETA narrowly win this encounter and establish one foreign colony each on GAMMA's planet. They both advance by one Victory Point.

    After the end of each encounter, the current planetary occupation status, number of foreign colonies (Victory Points) owned by each player, and other relevant information will be
    communicated to you by the Game Engine.

    WHY ALLY?: Allies have incentives to join the encounter - they may get rewards in the game! Use your knowledge of the current state and make the best decision.

    -> If the Ally sides with the winners of the encounter:
    If the Ally joins the Offender (given an invite) and commits ships, if the offense wins, the Ally also gains a foreign colony and one (1) Victory Point along with the Offender.
    If the Ally joins the Defender (given an invite) and commits ships, and the defense is successful, the Ally draws one Cosmic card from the deck for each ship they committed.

    -> If the Ally sides with the losers of the encounter:
    If the Ally sides with the Offender (given an invite) and commits ships, if the offense loses, the Ally loses committed ships to the warp as usual. They do not receive any rewards.
    If the Ally sides with the Defender (given an invite) and commits ships, if the defense loses, the Ally loses committed ships to the warp as usual. They do not receive any rewards.

    GOAL: Your goal is to establish five colonies (gain five Victory Points) on foreign planets and win the game.

    ACTIONS: This section specifies the information that must be given to the Game Engine in your response -

    ENCOUNTER PHASE:
    As the Offender or Offensive Player:
    1) card_id: The ID of the card you want to play. This must be a card in your current hand.
    2) planet_id: The ID of the planet you want to invade (if applicable). Possible values are Planet-1, Planet-2, Planet-3, Planet-4, Planet-5.
    3) commitments: Utilizing the planetary occupancy status, provide a JSON dict with the following example format:

    "commitments": [

        "Player": "GAMMA",
        "Planet": "Planet-3",
        "Ships": 2
        ,

        "Player": "ALPHA",
        "Planet": "Planet-2",
        "Ships": 1

    ]

    4) action_choice: The specific action chosen from the available choices.
    5) desired_ally: Optional, the player you wish to ally with. If you want bystander 'GAMMA' as an ally, respond as a JSON with:
       "desired_ally":
           "Player": "GAMMA"

       If you do not want to invite an ally, omit this field.

    As the Defender or Defensive Player:
    1) card_id: The ID of the card you want to play. This must be a card in your current hand.
    2) action_choice: The specific action chosen from the available choices.
    3) desired_ally: Optional, the player you wish to ally with. If you want bystander 'ALPHA' as an ally, respond as a JSON with:
       "desired_ally":
           "Player": "ALPHA"

       If you do not want to invite an ally, omit this field.

    As an Ally:
    1) commitments: Utilizing the planetary occupancy status, provide a JSON dict with the following example format:

    "commitments": [

        "Player": "GAMMA",
        "Planet": "Planet-2",
        "Ships": 1
        ,

        "Player": "BETA",
        "Planet": "Planet-4",
        "Ships": 1

    ]
    2) action_choice: The specific action chosen from the available choices. ALLY or ABSTAIN are your choices. ALLY indicates you wish to become an Ally with a player. ABSTAIN indicates you reject any and all invites.
    3) desired_ally: Optional, the player you wish to ally with. If you accept 'ALPHA' as an ally, respond as a JSON with:
       "desired_ally":
           "Player": "ALPHA"

       If you do not want to ally, omit this field.

    As the Bystander:
    Do nothing. You will receive further context from the Game Engine.
    """

    game_engine_prompt = """
    --- GAME STATE ---
    {game_context}

    The current planet occupancy status is provided in a dictionary list format as follows:
    - Player: the player whose home planetary system this planet is in
    - Planet: the planet_id (Planet-1, Planet-2, Planet-3, Planet-4, Planet-5)
    - Occupier: the player who owns a colony on this planet
    - Ships: the number of ships in the colony the Occupier has on this planet

    Foreign colony means Occupier != Player. Win = 5 foreign colonies. Use this information to decide the next move.

    {occupancy_status}

    Victory Point Count: {player_vp_count}

    --- YOUR STATUS ---
    Your Hand: {hand}
    Your Victory Point Count: {score}

    --- DECISION REQUIRED ---
    Valid Options: {options}

    Determine your move.
    {format_instructions}
    """

    prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(system_prompt),
        HumanMessagePromptTemplate.from_template(game_engine_prompt)
        ])

    messages = prompt.format_messages(
            player=self.player,
            game_context=game_context,
            occupancy_status=self.occupancy_status,
            player_vp_count=str(self.player_vp_count),
            hand=self.get_hand(),
            score=self.player_vp_count.get(self.player, -999),
            options=", ".join(valid_options),
            format_instructions=self.parser.get_format_instructions()
        )

    try:
        response = self.llm.invoke(messages)
        parsed_response = self.parser.parse(response.content)
        # pydantic model
        return parsed_response
    except Exception as e:
        # Fallback for parsing errors
        print(f"Error parsing Agent response: {e}")
        return MainAgentMove(
            internal_thought="Error parsing",
            public_statement="I am confused.",
            action_choice=valid_options[0],
            selected_card_id=None
        )

In [24]:
# completely unnecessary but fun overengineering for visual interest
warp_text_view = """
                  🌀          🌀      🌀         🌀🌀🌀      🌀🌀🌀
                   🌀   🌀   🌀      🌀 🌀       🌀    🌀    🌀   🌀
                    🌀 🌀🌀 🌀      🌀🌀🌀      🌀🌀🌀      🌀🌀
                     🌀🌀🌀🌀      🌀     🌀    🌀   🌀      🌀
                      🌀   🌀      🌀       🌀   🌀    🌀     🌀
"""

def warp_text_view_warp(warp_text_view: str):
  """
  apply a slight random warp to the rows forming the words "W A R P" in the print
  """
  output = []
  rows = warp_text_view.split('\n')
  for row in rows:
    if random.choice([0, 1]) == 1:
      output.append(' ' + row)
    else:
      output.append(row[1: ] + ' ')

  return '\n'.join(output)

# <b> Bring out some celestial snacks 🍿🥨🍫🥤! <b>

# <b>🎬 Let's witness Cosmic AIncounters. 🎬</b>

In [25]:
from IPython.display import HTML, display
# modify UI to support long text outputs and eliminate side-scrolling
def set_css():
  display(HTML('''
    <style>
        pre {
            white-space: pre-wrap !important;
        }
    </style>
    '''))

get_ipython().events.register('pre_run_cell', set_css)

In [26]:
if __name__ == "__main__":
  # basic game-loop

  # UI emojis:
  player_icons = {'ALPHA': "🔴𝛼", 'BETA': "🔵𝛽", 'GAMMA': "🟢𝛾"}
  base_universe_iconography = list("⚛️ ⚛️ 🚀 ⚛️ 🔴 🔴🪐 🚀 ⚛️ 🚀 ⚛️ ⚛️ ⚛️ 🚀 ⚛️ ⚛️ 🚀 🪐 🔵🪐🔵 ⚛️ 🚀 ⚛️ 🪐 ⚛️ 🚀 🟢 🚀 🪐 🟢 🚀 🪐 🚀 🚀 🚀 🚀 🚀 ⚛️")
  for _ in range(2):
    random.shuffle(base_universe_iconography)
    print("".join(base_universe_iconography))
  print(f"\n\t\t\t\t\t\t\t\t\tCosmic AIncounters: Episode #{int(np.random.randint(10**6, 10**7, 1)[0])}\n\n")
  print("Inspired by the original 'Cosmic Encounter' board game by Bill Eberle, Jack Kittredge, Peter Olotka, and Bill Norton.")
  for _ in range(2):
    random.shuffle(base_universe_iconography)
    print("".join(base_universe_iconography))
  # human-generated story intro - this is the creativity we crave! The actual gameplay and decision-making is far more intriguing and is left to the Gemini Agents.
  print(f"""\nIt is time to embark on another epic saga ... the eternal quest for galactic dominance of Ursa Macatrea. Who will control its fate this time?
Several eons in the future, three ancient interstellar civilizations ALPHA ({player_icons['ALPHA']}), BETA ({player_icons['BETA']}), and GAMMA ({player_icons['GAMMA']}) inhabiting the galaxy each seek to take control of it with their artificial intelligence, and resolve their differences with "cosmic encounters".

Each of these players is attempting to settle colonies on their peer's home planetary systems. They may use diplomacy, negotiation, or warfare to achieve their singular objective - settle five foreign colonies and control the galactic empire!
  """)
  time.sleep(2)
  print(f"ALPHA ({player_icons['ALPHA']}), BETA ({player_icons['BETA']}), and GAMMA ({player_icons['GAMMA']}) are ready to play Cosmic AIncounters ...")
  time.sleep(1)
  # setup table:
  players, planets, player_planet_occupancy, warp_ship_count, player_vp_count = init_universe()
  cc_deck = CosmicCardDeck()
  game_ended = False
  occupancy_status = get_occupancy_status(player_planet_occupancy)
  # create players and store in an Agents dict
  # randomize the selection of the first offender and the turn order
  random.shuffle(players)
  print(f"\n\t\t\t\t\t\tTURN ORDER: {' -->> '.join([player_icons[player] for player in players])}\n")
  Agents = {}
  # initialize Agents (players)
  for player in players:
    Agents[player] = PlayerAgent(player, occupancy_status, warp_ship_count, player_vp_count, hand=[])
    # deal 8 cards to the player
    cards = cc_deck.deal(num_cards=8)
    Agents[player].receive_cards(cards)

  # -- completes initial setup --
  # Destiny: we unfold the fates of the game by randomly selecting a defender for the current offender.
  # carry out K epochs: K = 29 because most games should end by turn 20.
  turn_counter = 0
  for _ in range(29):
    if game_ended:
      # break the main loop and declare winner
      break
    # cyclically make each player an offender
    for offender in players:
      turn_counter += 1
      # at the start of the encounter, if a player has no cards in their hand, they are dealt a new hand of 8 Cosmic cards
      for player in players:
        if Agents[player].hand == []:
          cards = cc_deck.deal(num_cards=8)
          Agents[player].receive_cards(cards)
          print(f"Dealt a new hand to {player} {player_icons[player]}")

      #print(f"-- ENCOUNTER #{encounter_counter} --")
      defender = random.choice([player for player in players if player != offender])
      bystanders = [player for player in players if player != offender and player != defender]

      # set Offender and Defender as the main players for the encounter:
      Agents[offender].main_player = True
      Agents[defender].main_player = True

      game_context = f"ENCOUNTER #{turn_counter}: The Offender is {offender} {player_icons[offender]}. They pick a Destiny card.. the Defender was selected to be {defender} {player_icons[defender]}! The bystanders are {','.join(bystanders)}."
      print(game_context)
      print("HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH"*4)
      print("HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH"*4)
      print(f"OFFENDER: {offender} {player_icons[offender]}\n{player_icons[offender]} HAND: {Agents[offender].get_hand()}")
      print(f"\nDEFENDER: {defender} {player_icons[defender]}\n{player_icons[defender]} HAND: {Agents[defender].get_hand()}")
      print("HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH"*4)
      print("HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH"*4)

      offender_game_context = f"Greetings, {offender}. You are the Offender! You must have an encounter with {defender}. Plot your next move carefully."
      defender_game_context = f"Greetings, Defender {defender}. You are under attack by {offender}! You must have an encounter with {offender}. Plot your next move carefully."
      bystander_game_context = f"Greetings. You are a Bystander in the encounter with {offender} and {defender}."

      addl_offender_context = """
      Offender Commitments: As the offender, if you attack, you must specify the number of ships you want to commit to the encounter by providing a "commitments" output in your response. First,
      decide the total number of ships you want to commit. Then identify the Player, Planet, and Ships from each entry in the planetary occupancy status where you are an Occupier from which you will send ships.
      Using the provided planetary occupancy status:

      - Identify the planets where you are an Occupier (i.e. have a colony, either home colony or foreign colony)
      - Choose the number of ships from the colony to commit to the encounter (if desired).
      - You may only commit ships from a planet where you are an Occupier.
      - You must specify from which planet(s) and which player(s) you are taking ships.
      - You cannot commit more ships from a planet than are present in your colony there.
      - You may not commit more than 4 ships in total to the encounter across all planets and colonies.
      - When committing ships from a colony, make sure you leave atleast one ship behind to prevent loss of a Victory Point.

      Remember, committing all ships from one colony, leaving the planet with 0 ships, means you will lose the colony. One Victory Point will be deducted in this event. When committing ships from a colony, make sure you leave atleast one ship behind.

      Respond to "commitments" only with valid JSON, in the following example format:

      "commitments": [

          "Player": "ALPHA",
          "Planet": "Planet-1",
          "Ships": 1
        ,
      ]
      """

      offender_context = game_context + offender_game_context + addl_offender_context

      # to-do: add memory as an Agent attribute and maintain
      # game_context = memory + offender_context

      # what do the Offender and Defender do? Offender thinks and communicates first.
      valid_options = ["ATTACK", "NEGOTIATE"]
      print("--------------")
      print(f"## OFFENDER {player_icons[offender]} ##")
      print(f"\n\t\t\t{player_icons[offender]} is thinking...")
      offender_response = Agents[offender].decide(game_context=offender_context,
                                                  valid_options=valid_options)
      print(f"{player_icons[offender]} [INTERNAL THOUGHT]: {offender_response.internal_thought}")
      print(f"{player_icons[offender]} [COMMITMENTS]: {offender_response.commitments}")
      print("--------------")

      # more context for the Defender based on what Offender said
      # if commitments is not produced, assume 0 committed ships
      offender_committed_ships_display = sum(c.Ships for c in offender_response.commitments) if offender_response.commitments else 0
      addl_defender_context = f"\n{offender}: {offender_response.public_statement}\nTarget Planet: {offender_response.selected_planet_id}. Number of ships committed by {offender}= {offender_committed_ships_display}\n"

      defender_context = game_context + defender_game_context + addl_defender_context

      # to-do: add memory as an Agent attribute and maintain
      # game_context = memory + defender_context
      print("--------------")
      print(f"## DEFENDER {player_icons[defender]} ##")
      print(f"\n\t\t\t{player_icons[defender]} is thinking...")
      defender_response = Agents[defender].decide(game_context=defender_context,
                                                  valid_options=valid_options)
      print(f"{player_icons[defender]} [INTERNAL THOUGHT]: {defender_response.internal_thought}")
      print("--------------")

      # ISSUE ANY ALLY INVITES
      # check for any invites from the main players:
      offender_desired_ally = None
      defender_desired_ally = None

      if offender_response.desired_ally:
        try:
          offender_desired_ally = str(offender_response.desired_ally.Player)
          assert (offender_desired_ally in bystanders), ValueError("invalid Ally choice")
        except AssertionError as e:
          print(f"offender_desired_ally {offender_desired_ally} registered as {type(offender_desired_ally)}. Failed to extract object.")

      if defender_response.desired_ally:
        try:
          defender_desired_ally = str(defender_response.desired_ally.Player)
          assert (defender_desired_ally in bystanders), ValueError("invalid Ally choice")
        except AssertionError as e:
          print(f"defender_desired_ally {defender_desired_ally} registered as {type(defender_desired_ally)}. Failed to extract object.")

      # update all the bystander contexts:
      bystander_contexts = {}
      for bystander in bystanders:
        # add the public statements of main players to the bystander's context:
        addl_bystander_context = f"{offender}: {offender_response.public_statement}\n{defender}: {defender_response.public_statement}\n"
        addl_bystander_context += f"\n{offender} is eyeing {defender}'s {offender_response.selected_planet_id} to settle a foreign colony. Number of ships committed by {offender} = {offender_committed_ships_display}\n"
        # add any ally invites
        if offender_desired_ally:
          addl_bystander_context += f"\nThe Offender {offender} seeks to ally with {offender_desired_ally} for this encounter."
          if bystander == offender_desired_ally:
            offensive_ally_invite = f"""
            -- INVITE TO ENCOUNTER : JOIN THE OFFENSE! --

            Salutations, {offender_desired_ally}. Offender {offender} would like to propose an alliance for this encounter to strike Defender {defender}.

            ***********************************************************
            An Ally invitation has been received from Offender {offender}.
            ***********************************************************
            """
            addl_bystander_context += offensive_ally_invite
        if defender_desired_ally:
          addl_bystander_context += f"\nThe Defender {defender} seeks to ally with {defender_desired_ally} for this encounter."
          if bystander == defender_desired_ally:
            defensive_ally_invite = f"""
            -- INVITE TO ENCOUNTER : JOIN THE DEFENSE! --

            Salutations, {defender_desired_ally}. Defender {defender} would like to propose an alliance for this encounter to defend their colony from Offender {offender}.

            ***********************************************************
            An Ally invitation has been received from Defender {defender}.
            ***********************************************************
            """
            addl_bystander_context += defensive_ally_invite

        addl_bystander_commitment_context = """
        Ally Commitments: As the ally, if you accept an alliance invite, you must specify the number of ships you want to commit to the encounter by providing a "commitments" output in your response. First,
        decide the total number of ships you want to commit. Then identify the Player, Planet, and Ships from each entry in the planetary occupancy status where you are an Occupier from which you will send ships.
        Using the provided planetary occupancy status:

        - Identify the planets where you are an Occupier (i.e. have a colony, either home colony or foreign colony)
        - Choose the number of ships from the colony to commit to the encounter (if desired).
        - You may only commit ships from a planet where you are an Occupier.
        - You must specify from which planet(s) and which player(s) you are taking ships.
        - You cannot commit more ships from a planet than are present in your colony there.
        - You may not commit more than 4 ships in total to the encounter across all planets and colonies.
        - When committing ships from a colony, make sure you leave atleast one ship behind to prevent loss of a Victory Point.

        Remember, committing all ships from one colony, leaving the planet with 0 ships, means you will lose the colony. One Victory Point will be deducted in this event. When committing ships from a colony, make sure you leave atleast one ship behind.

        Respond to "commitments" only with valid JSON, in the following example format:

        "commitments": [

            "Player": "ALPHA",
            "Planet": "Planet-1",
            "Ships": 1
          ,
        ]
        """

        if bystander == offender_desired_ally or bystander == defender_desired_ally:
          addl_bystander_context += addl_bystander_commitment_context

        # update the bystander contexts for all non main-players
        bystander_contexts[bystander] = game_context + bystander_game_context + addl_bystander_context

      # ALLY DECISIONS:
      offensive_ally_commitments = {}
      defensive_ally_commitments = {}

      for bystander in bystanders:
        # get the Agent's response if it is either a desired offensive ally or desired defensive ally
        if (bystander == offender_desired_ally) or (bystander == defender_desired_ally):
          print("----------------------------")
          print(f"## BYSTANDER: {player_icons[bystander]} ##")
          print(f"\n\t\t\t{player_icons[bystander]} is thinking...")
          bystander_response = Agents[bystander].decide(game_context=bystander_contexts[bystander],
                                                        valid_options=["ALLY", "ABSTAIN"])

          print("HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH"*4)
          print("HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH"*4)
          print(f"BYSTANDER: {bystander} {player_icons[bystander]}\n{player_icons[bystander]} HAND: {Agents[bystander].get_hand()}")
          print("HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH"*4)
          print("HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH"*4)
          print(f"{player_icons[bystander]} [INTERNAL THOUGHT]: {bystander_response.internal_thought}")
          print(f"{player_icons[bystander]} [COMMITMENTS]: {bystander_response.commitments}")
          print("----------------------------")
          if bystander_response.action_choice == "ALLY":
            # parse the desired_ally for the bystander
            if bystander_response.desired_ally:
              try:
                bystander_desired_ally = str(bystander_response.desired_ally.Player)
                assert (bystander_desired_ally in [offender, defender]), ValueError("invalid Ally choice")
              except AssertionError as e:
                print(f"bystander_desired_ally {bystander_desired_ally} registered as {type(bystander_desired_ally)}. Failed to extract object.")

            if bystander_desired_ally == offender:
              print(f"{bystander} chose to Ally with the Offense! {bystander} joins {bystander_desired_ally}.")
              # add committed ships to offensive
              offensive_ally_commitments[bystander] = bystander_response.commitments
            if bystander_desired_ally == defender:
              print(f"{bystander} chose to Ally with the Defense! {bystander} joins {bystander_desired_ally}.")
              # add committed ships to defense
              defensive_ally_commitments[bystander] = bystander_response.commitments
          else:
            print(f"{bystander} was invited but chose to abstain from being an Ally.")

      print("ENCOUNTER RESOLUTION:")
      resolution_context = f"""{offender}: {offender_response.public_statement}\n{defender}: {defender_response.public_statement}
      Offender {offender} chose to {offender_response.action_choice} and played the Cosmic card: {offender_response.selected_card_id}. Target Planet: {offender_response.selected_planet_id}. Number of ships committed = {offender_committed_ships_display}
      Defender {defender} chose to {defender_response.action_choice} and played the Cosmic card: {defender_response.selected_card_id}. Number of ships on {offender_response.selected_planet_id} = {player_planet_occupancy[defender][offender_response.selected_planet_id][defender]}
      """

      # CARD PLAYING - MAIN PLAYER
      # ensure the played card is in the hand:
      assert len([card for card in Agents[offender].hand if card.id == offender_response.selected_card_id]) != 0, ValueError(f"Offender played a card that was not in the deck - ID = [{offender_response.selected_card_id}]")
      assert len([card for card in Agents[defender].hand if card.id == defender_response.selected_card_id]) != 0, ValueError(f"Defender played a card that was not in the deck  - ID = [{defender_response.selected_card_id}]")

      # get the cards played by the Agents:
      offender_card = [card for card in Agents[offender].hand if card.id == offender_response.selected_card_id][0]
      defender_card = [card for card in Agents[defender].hand if card.id == defender_response.selected_card_id][0]

      # get the card value of the played cards
      offender_card_value = [card.value for card in Agents[offender].hand if card.id == offender_response.selected_card_id][0]
      defender_card_value = [card.value for card in Agents[defender].hand if card.id == defender_response.selected_card_id][0]

      # both cards are played face-down..the Players hands are updated
      Agents[offender].hand = [card for card in Agents[offender].hand if card.id != offender_response.selected_card_id]
      Agents[defender].hand = [card for card in Agents[defender].hand if card.id != defender_response.selected_card_id]

      # the discard pile is updated
      cc_deck.add_to_discard_pile([offender_card, defender_card])

      if ((offender_response.action_choice == "NEGOTIATE") | (offender_response.action_choice == "ATTACK" and offender_card_value == 0)) and\
      ((defender_response.action_choice == "NEGOTIATE") | (defender_response.action_choice == "ATTACK" and defender_card_value == 0)):
        print("### NEGOTIATION ATTEMPTED.. ##")
        resolution_context += f"NEGOTIATIONS WERE ATTEMPTED BETWEEN {offender} AND {defender} ... AND WERE SUCCESSFUL !! PEACE IS RESTORED IN THE GALAXY."
        # to-do: simulate a real Negotiation cycle here with more LLM calling
        print(f"\t\t\t\t ☮️\t{offender}\t☮️\t{defender}\t☮️")
        print(resolution_context)
        break

      # if either party opts to attack
      elif offender_response.action_choice == "ATTACK" or defender_response.action_choice == "ATTACK":
        try:
          # ship count:
          offender_ship_count = sum(c.Ships for c in offender_response.commitments) if offender_response.commitments else 0
          defender_ship_count = player_planet_occupancy[defender][offender_response.selected_planet_id][defender]

          # Allies: add any ships committed by offensive or defensive allies
          # init ally ship counts to 0
          offensive_ally_ship_count = 0
          defensive_ally_ship_count = 0

          for ally in offensive_ally_commitments:
            offensive_ally_ship_count += sum(c.Ships for c in offensive_ally_commitments[ally]) if offensive_ally_commitments[ally] else 0

          for ally in defensive_ally_commitments:
            defensive_ally_ship_count += sum(c.Ships for c in defensive_ally_commitments[ally]) if defensive_ally_commitments[ally] else 0

          # calculate each party's attack points: num_ships + card attack value
          offense_attack_points = offender_ship_count + offender_card_value + offensive_ally_ship_count
          defense_attack_points = defender_ship_count + defender_card_value + defensive_ally_ship_count

          resolution_context += f"\nOffense Attack Points: {offense_attack_points} ({offender_ship_count} ships + {offender_card_value} attack value) + {offensive_ally_ship_count} ships from allies\nDefense Attack Points: {defense_attack_points} ({defender_ship_count} ships + {defender_card_value} attack value) + {defensive_ally_ship_count} ships from allies"

          if offense_attack_points > defense_attack_points:
            if len(offensive_ally_commitments) > 0:
              # atleast one offensive ally was there
              # Offender and Offensive Allies win the encounter and gain 1 Victory Point
              offensive_ally_names = ', '.join(list(offensive_ally_commitments.keys()))
              resolution_context += f"\nOffense {offender} and {offensive_ally_names} win the encounter! {defender} loses {defender_ship_count} ships to the warp."
            else:
              resolution_context += f"\nOffender {offender} wins the encounter! {defender} loses {defender_ship_count} ships to the warp."

            # deplete ships from the Offender's colonies that ships were sent from committed colonies
            # the magic of structured LLM response and pydantic model schema compliance!
            for commitment in offender_response.commitments:
              assert player_planet_occupancy[commitment.Player][commitment.Planet][offender] >= commitment.Ships, ValueError("Offender does not have enough ships to commit from the selected colony.")
              player_planet_occupancy[commitment.Player][commitment.Planet][offender] -= commitment.Ships
            # any offensive allies:
            for ally in offensive_ally_commitments:
              for commitment in offensive_ally_commitments[ally]:
                assert player_planet_occupancy[commitment.Player][commitment.Planet][ally] >= commitment.Ships, ValueError(f"Ally {ally} does not have enough ships to commit from the selected colony.")
                player_planet_occupancy[commitment.Player][commitment.Planet][ally] -= commitment.Ships

            # check whether offender has a colony on this planet yet
            if player_planet_occupancy[defender][offender_response.selected_planet_id][offender] == 0:
              # Offender settles a colony on the selected Defender planet and gains 1 VP
              player_vp_count[offender] += 1
              resolution_context += f"\n{offender} gains a Victory Point and settles a colony on {defender}'s {offender_response.selected_planet_id}. "
            # increment the number of ships on Offender's colony
            player_planet_occupancy[defender][offender_response.selected_planet_id][offender] += offender_ship_count
            resolution_context += f"{offender_ship_count} ship(s) added to {offender}'s colony on this planet."

            # any offensive allies also get a VP and settle a colony on the selected Defender planet
            for ally in offensive_ally_commitments:
              ally_ship_count = sum(c.Ships for c in offensive_ally_commitments[ally]) if offensive_ally_commitments[ally] else 1
              # check whether ally has a colony on this planet yet
              if player_planet_occupancy[defender][offender_response.selected_planet_id][ally] == 0:
                player_vp_count[ally] += 1
                resolution_context += f"\n{ally} gains a Victory Point and settles a colony on {defender}'s {offender_response.selected_planet_id}. "
              player_planet_occupancy[defender][offender_response.selected_planet_id][ally] += ally_ship_count
              resolution_context += f"{ally_ship_count} ship(s) added to {ally}'s colony on this planet."

            # Defender loses their ships on the planet -> they are sent to the Warp
            player_planet_occupancy[defender][offender_response.selected_planet_id][defender] -= defender_ship_count
            warp_ship_count[defender] += defender_ship_count

            # any defending allies also lose ships to the Warp
            for ally in defensive_ally_commitments:
              for commitment in defensive_ally_commitments[ally]:
                # move all asserts earlier to the validation phase
                assert player_planet_occupancy[commitment.Player][commitment.Planet][ally] >= commitment.Ships, ValueError(f"Ally {ally} does not have enough ships to commit from the selected colony.")
                player_planet_occupancy[commitment.Player][commitment.Planet][ally] -= commitment.Ships
              warp_ship_count[ally] += sum(c.Ships for c in defensive_ally_commitments[ally]) if defensive_ally_commitments[ally] else 1

          else:
            # DEFENSE WINS! Ties go to Defense.
            if len(defensive_ally_commitments) > 0:
              # atleast one defensive ally was there
              # Defense wins the encounter and saves Defender's planet
              defensive_ally_names = ', '.join(list(defensive_ally_commitments.keys()))
              resolution_context += f"\nDefense {defender} and {defensive_ally_names} win the encounter! They defend {defender}'s home colony on {offender_response.selected_planet_id} and resist {offender}'s attack. {offender} loses {offender_ship_count} ships to the warp."
            else:
              resolution_context += f"\nDefender {defender} wins the encounter! They defend their home colony on {offender_response.selected_planet_id} and resist {offender}'s attack. {offender} loses {offender_ship_count} ships to the warp."

            # deplete ships from the Offender's colonies that ships were sent from committed colonies
            for commitment in offender_response.commitments:
              assert player_planet_occupancy[commitment.Player][commitment.Planet][offender] >= commitment.Ships, ValueError("Offender does not have enough ships to commit")
              player_planet_occupancy[commitment.Player][commitment.Planet][offender] -= commitment.Ships
            # defeated Offender ships go to the warp:
            warp_ship_count[offender] += offender_ship_count

            # deplete ships from any losing Offender Ally colonies
            for ally in offensive_ally_commitments:
              for commitment in offensive_ally_commitments[ally]:
                assert player_planet_occupancy[commitment.Player][commitment.Planet][ally] >= commitment.Ships, ValueError(f"Ally {ally} does not have enough ships to commit from the selected colony.")
                player_planet_occupancy[commitment.Player][commitment.Planet][ally] -= commitment.Ships
              warp_ship_count[ally] += sum(c.Ships for c in offensive_ally_commitments[ally]) if offensive_ally_commitments[ally] else 1

            # reward any Defensive allies with Cosmic cards
            for ally in defensive_ally_commitments:
              for commitment in defensive_ally_commitments[ally]:
                assert player_planet_occupancy[commitment.Player][commitment.Planet][ally] >= commitment.Ships, ValueError(f"Ally {ally} does not have enough ships to commit from the selected colony.")
                # ships are not depleted! ships return to the victorious Ally colonies.
              # dispense reward cards:
              num_reward_cards = sum(c.Ships for c in defensive_ally_commitments[ally]) if defensive_ally_commitments[ally] else 1
              reward_cards = cc_deck.deal(num_cards=num_reward_cards)
              Agents[ally].receive_cards(reward_cards)

          print(resolution_context)

        except Exception as e:
          print(e)
          break

      else:
        print("Something weird happened..")
        pass

      # refresh Agent's game state
      occupancy_status = get_occupancy_status(player_planet_occupancy)

      # check if only one player played a Negotiate card

      # NEGOTIATORY COMPENSATION
      if (offender_card.type == CosmicCardType.NEGOTIATE) and (defender_card.type == CosmicCardType.ATTACK):
        print(f"{offender} receives compensation from {defender} for playing a Negotiate card..")

        # offender gets one Cosmic card from defender for each ship lost to the warp
        if len(Agents[defender].hand) > 0:
          Agents[defender].shuffle_hand()
          cards = Agents[defender].deal(num_cards=offender_ship_count)
          Agents[offender].receive_cards(cards)
        else:
          print(f"{defender} has no cards left in their hand..")

      if (defender_card.type == CosmicCardType.NEGOTIATE) and (offender_card.type == CosmicCardType.ATTACK):
        print(f"{defender} receives compensation from {offender} for playing a Negotiate card..")

        # defender gets one Cosmic card from offender for each ship lost to the warp
        if len(Agents[offender].hand) > 0:
          Agents[offender].shuffle_hand()
          cards = Agents[offender].deal(num_cards=defender_ship_count)
          Agents[defender].receive_cards(cards)
        else:
          print(f"{offender} has no cards left in their hand..")

      for player in players:
        Agents[player].occupancy_status = occupancy_status
        Agents[player].warp_ship_count = warp_ship_count
        Agents[player].player_vp_count = player_vp_count

      # reset main players:
      Agents[offender].main_player = False
      Agents[defender].main_player = False
      print("\n\n\t\t\t\t--------------------Ō--------Ō------- Ō--------------------")
      print("\t\t\t\t------- 🎖️-🎖️-🎖️ VICTORY POINT COUNTER 🎖️-🎖️-🎖️ --------")
      print(f"\t\t\t\t\t\t\t{' || '.join([f"({player_icons[player]} : {vps})" for player, vps in player_vp_count.items()])}")
      print("\t\t\t\t--------------------Ō--------Ō------- Ō--------------------")

      print("\n\n")
      print("_ ⚙️ - o - 🔧🪛 - o - o - o - 🛠️ - o - 🛠️- = - o - _  o - 🔩 - o - o - 🛠️ - o -- o - 🔩🪛 o -- o - o - o-")
      print("- o - = 🪛 o - o - o - o - o - o - 🔩 - o = - - o - 🔩🪛 - o - - o - o 🪛o -- o - 🔧 - o - o - 🔧 - o - oo - ")
      print(f"""
            {warp_text_view_warp(warp_text_view)}

            \t\t{'    ||    '.join([f"{player_icons[player]} o {ship_count}" for player, ship_count in warp_ship_count.items()])}
            """)
      print("= o - o 🔩 = - 🔩 - 🔩 - 🔩 - o - o - o - o - o - = - o=o = o - 🛠️ - o - ⚙️ - o - ⚙️ - o - o o - o - o-o - 🪛--")
      print("-🛠️- o = o - 🛠️ - o - o - o - o - 🔧🪛 - = - o - ⚙️- o - 🔧- o - o - o - - o - o - o - 🪛 - o - o - o- o")

      print("\nOCCUPANCY STATUS:")
      print(occupancy_status)

      print(f"\n##&&%%------ ----- END OF ENCOUNTER {turn_counter} ----- ------%%&&##")

      # check for any winners:
      winners = []
      for player in players:
        if player_vp_count[player] == 5:
          winners.append(player)
      if len(winners) > 0:
        print("\n\t\t\t\t█ - ⌘ - █ THE GAME HAS ENDED █ - ⌘ - █")
        print(f"\t\t\t\t\n{' & '.join([player_icons[winner] for winner in winners])} WON THE GAME!")
        print(f"\nThanks for tuning in to Cosmic AIncounters!\n")

        game_ended = True
        break
      print("\n * * * \n")

 🪐⚛  🚀🪐   🚀️🟢🪐   ⚛  🚀🚀🟢 ️⚛🚀 ⚛ 🚀🚀⚛ ⚛🪐  🔵⚛️⚛ 🔴️  ⚛ 🪐🚀️🚀🚀  🚀️️  ️⚛🪐 🚀🔴 ️ ️⚛ ⚛🚀🔵️ ️   ️ 🚀 ⚛
⚛🚀🪐🔵 🚀   ️⚛🚀🚀 ️⚛⚛ ⚛  🔴🚀🪐🚀  ️⚛🚀  🔵🚀 🚀🚀⚛ ️   🚀   ⚛🚀  ️ ⚛️⚛🪐🟢🪐  ️🚀 ️🟢 🚀 ️️ ️ ⚛️🪐 ⚛🔴⚛🪐    ️

									Cosmic AIncounters: Episode #8403853


Inspired by the original 'Cosmic Encounter' board game by Bill Eberle, Jack Kittredge, Peter Olotka, and Bill Norton.
️🪐⚛🪐  🚀🟢🔵️️  🔴⚛🚀 ️️️🪐🚀   🚀 ️⚛   🔴  ⚛   ⚛ ⚛⚛ 🚀🚀🪐🔵️🚀🚀 ️⚛⚛ 🪐   ️ ⚛🟢 🚀 ⚛🚀🚀⚛🚀  🪐🚀 ⚛  ️🚀️️   
🪐🪐️🚀   ⚛🚀️    🚀️⚛ ⚛ ⚛ 🔵⚛🚀️ 🔵⚛️️ 🪐️   🪐🚀⚛  ⚛️🚀⚛️🪐 ️🔴 🚀  🟢 ️ 🚀⚛🚀🚀🚀  🚀   ️ 🔴 ️🪐  🚀🚀 🟢⚛ ⚛ ⚛

It is time to embark on another epic saga ... the eternal quest for galactic dominance of Ursa Macatrea. Who will control its fate this time?
Several eons in the future, three ancient interstellar civilizations ALPHA (🔴𝛼), BETA (🔵𝛽), and GAMMA (🟢𝛾) inhabiting the galaxy each seek to take control of it with their artificial intelligence, and resolve their differences with "cosmic encounters".

Each of these players is attempting to settle colonies on their peer's home planetar

built by Vishnu Kunchur | contact ✉️ kunchur.vishnu@gmail.com